<a href="https://colab.research.google.com/github/e18679/ML0930/blob/master/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",   
    extract=True,
)

In [2]:
import glob
# glob.glob("/root/.keras/datasets/aclImdb/train/pos/*")
import os
import pandas as pd
def read(path):
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()
    return content

def get_data(t):
    dn = os.path.dirname(dataset)
    pattern = os.path.join(dn, "aclImdb", t, "pos", "*.txt")
    pos = glob.glob(pattern)
    pattern = os.path.join(dn, "aclImdb", t, "neg", "*.txt")
    neg = glob.glob(pattern)
    sentiments = [1] * len(pos) + [0] * len(neg)
    contents = map(read, pos + neg)
    df = pd.DataFrame({
        "contents":contents,
        "sentiment":sentiments
    })
    return df

train_df = get_data("train")
test_df = get_data("test")

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
# fit_on_texts: fit_transform 的 fit
tok.fit_on_texts(train_df["contents"])
# seq: 把所有單字換成數字的序列
x_train_seq = tok.texts_to_sequences(train_df["contents"])
x_test_seq = tok.texts_to_sequences(test_df["contents"])

In [4]:
# tok.word_index
tok.index_word[2]

'and'

In [5]:
# pd.DataFrame(train_df_seq)
# truncating: pre 12345 -> 45 post 12345 -> 12
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, 512)
x_test_pad = pad_sequences(x_test_seq, 512)
pd.DataFrame(x_test_pad)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2397,1,17,70,31,4,480,408,2,3,224,4,8,9,9,90,1,17,50,218,46,81,1663,9,5,2523,18,10,89,1038,543,2523,45,50,450,134,1560,501,6,70
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,10,13,8,1,99,47,68,30,219,155,1471,117,1,556,4,8,47,146,9,2,33,506,9,44,33,506,9,10,101,22,77,5,10,199,9,3,787,43,4,155
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,5,25,2,88,239,77,2413,1,61,28,12,10,7,7,15,69,42,3,212,107,240,4,17,84,102,16,439,687,2,3,173,4,180,5,101,41,100,1,17,127
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2225,4,3,110,402,10,358,5,409,18,71,30,1,127,22,78,21,207,1173,2,8,58,476,1582,86,9,13,10,5,1,17,1,83,55,10,293,9,20,11,19
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,2697,4,1,153,2,109,196,176,243,3,608,4,102,365,2,23,942,5,1,62,206,36,91,290,1148,5,409,6,5,116,18,29,180,212,1346,1458,110,268,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,12,59,25,74,345,1,270,2,1,81,302,26,573,2624,482,2,46,2469,789,258,47,6,3,49,17,5,27,90,41,11,2197,1853,4,1,1206,587,11,6,21,9
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,129,6,3,2,1,325,247,34,805,15,87,6,3,2414,38,333,6,1,246,281,10,25,123,107,10,241,21,249,1,1415,385,4,482,67,866,94,53,15,1,581
24997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1225,318,598,4,11,187,163,53,15,1,2559,4,1,19,7,7,9,149,57,25,98,1104,42,40,75,21,75,21,35,75,42,49,40,2067,75,3,19,5,795,2
24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,816,42,217,37,10,64,3,15,1,1862,2,1,17,20,1,82,496,44,22,37,146,2277,99,394,31,1,330,11,235,27,9,58,59,92,27,3,738,1153,467


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, Flatten, Dropout

layers = [
  # 1~3000(token) + 0(padding)
  # params: 3001 * 128
  Embedding(3001, 128, mask_zero=True, input_length=512),
  Flatten(),
  Dense(128, activation="relu"),
  Dropout(0.25),
  Dense(2, activation="softmax")  
]
model = Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 512, 128)          384128    
_________________________________________________________________
flatten (Flatten)            (None, 65536)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               8388736   
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
Total params: 8,773,122
Trainable params: 8,773,122
Non-trainable params: 0
_________________________________________________________________


In [7]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer=Adam(),
       metrics=["accuracy"])

In [8]:
y_train = train_df["sentiment"]
y_test = test_df["sentiment"]

In [9]:
# epoch: 整份資料要看幾遍
# 60000筆, 2epoch, 200batch -> 60000 * 2 / 200次調整
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
    ModelCheckpoint("model.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)
]
model.fit(x_train_pad, 
     y_train,
     batch_size=100,
     epochs=50,
     validation_split=0.1,
     callbacks=callbacks,
     verbose=2)

Epoch 1/50
225/225 - 36s - loss: 0.4826 - accuracy: 0.7558 - val_loss: 0.3880 - val_accuracy: 0.8304
Epoch 2/50
225/225 - 36s - loss: 0.1780 - accuracy: 0.9355 - val_loss: 0.3539 - val_accuracy: 0.8516
Epoch 3/50
225/225 - 36s - loss: 0.0432 - accuracy: 0.9885 - val_loss: 0.5497 - val_accuracy: 0.8232
Epoch 4/50
225/225 - 36s - loss: 0.0073 - accuracy: 0.9990 - val_loss: 0.7897 - val_accuracy: 0.8124
Epoch 5/50
225/225 - 36s - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.6276 - val_accuracy: 0.8540
Epoch 6/50
225/225 - 36s - loss: 7.2357e-04 - accuracy: 1.0000 - val_loss: 0.8509 - val_accuracy: 0.8248
Epoch 7/50
225/225 - 36s - loss: 3.1098e-04 - accuracy: 1.0000 - val_loss: 0.9059 - val_accuracy: 0.8252


In [10]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 13s 17ms/step - loss: 0.3169 - accuracy: 0.8706


[0.3169404864311218, 0.870639979839325]